In [1]:
import pandas as pd
from unidecode import unidecode
import numpy as np
import scipy.io
from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
import os

PATH_UPV = "./DatosExperimento1/UPV"

# Leer el archivo Excel en un DataFrame de pandas
df = pd.read_excel(PATH_UPV+'/Exp1_Observadores_UPV.xlsx')
    
if not os.path.exists('Resultados_UPV'):
    os.makedirs('Resultados_UPV')

# Crear un diccionario vacío para almacenar los resultados
voluntarios_upv = {}

# Iterar sobre cada fila del DataFrame
for idx, row in df.iterrows():
    # Tratamiento de nombre y apellidos para evitar que una misma persona se trate como diferente:
    # Quito espacios, mayusculas, minuscula s, etc
    nombre = unidecode(row['Nombre'].lower())
    apellidos = unidecode(row['Apellidos'].lower())
    fichero = row['Fichero']
    # Crear la clave del diccionario usando el nombre y apellidos
    clave = f"{nombre}-{apellidos}".replace(" ", "")
    
    # Agregar el valor de la columna "Fichero" a la lista correspondiente en el diccionario
    if clave in voluntarios_upv:
        voluntarios_upv[clave].append(fichero)
    else:
        voluntarios_upv[clave] = [fichero]
voluntarios_upv

{'voluntario_upv_1-_1': ['ResultsJNDsNoiseFlatBG20211129T112353.mat',
  'ResultsJNDsNoiseFlatBG20211129T120005.mat'],
 'voluntario_upv_2-_2': ['ResultsJNDsNoiseFlatBG20211129T133717.mat',
  'ResultsJNDsNoiseFlatBG20211129T140322.mat'],
 'voluntario_upv_3-_3': ['ResultsJNDsNoiseFlatBG20211129T163621.mat',
  'ResultsJNDsNoiseFlatBG20211129T171934.mat'],
 'voluntario_upv_4-_4': ['ResultsJNDsNoiseFlatBG20211130T105700.mat',
  'ResultsJNDsNoiseFlatBG20220111T121738.mat'],
 'voluntario_upv_5-_5': ['ResultsJNDsNoiseFlatBG20211201T114113.mat',
  'ResultsJNDsNoiseFlatBG20211210T120626.mat'],
 'voluntario_upv_6-_6': ['ResultsJNDsNoiseFlatBG20211202T140213.mat',
  'ResultsJNDsNoiseFlatBG20211202T142641.mat'],
 'voluntario_upv_7-_7': ['ResultsJNDsNoiseFlatBG20211203T102719.mat',
  'ResultsJNDsNoiseFlatBG20220124T090423.mat'],
 'voluntario_upv_8-_8': ['ResultsJNDsNoiseFlatBG20211210T101730.mat',
  'ResultsJNDsNoiseFlatBG20211210T112932.mat'],
 'voluntario_upv_9-_9': ['ResultsJNDsNoiseFlatBG20211210

In [2]:
def normcdf2(x, mu, sigma):
    '''
    Utilizo la función norm.cdf de la librería scipy.stats para calcular la función de distribución acumulada (CDF) 
    de la distribución normal, y luego dividimos el resultado por dos y sumamos 0.5 para ajustar el rango a [0,1].
    '''
    y = (norm.cdf(x, mu, sigma) + 1) / 2
    return y

In [3]:
def normcdf2_minimizer(X, params):
    # X incluye mu y sigma
    mu = X[0]
    sigma = X[1]

    # x/y valores para ajustar
    x = params['x']
    y = params['y']

    # los valores ajustados con mu y sigma dados
    yfit = normcdf2(x, mu, sigma)

    # cálculo del error
    err = np.sum((yfit - y)**2)

    return err

In [4]:
# Variables generales
ruidos = scipy.io.loadmat('ruidosExp1.mat')
ruidos = np.array(ruidos['ruidos'])
minX = np.min(ruidos[:, 2])
maxX = np.max(ruidos[:, 2])
LoopNumber = 16
numberObservations = 10
    
for voluntario, ficheros in voluntarios_upv.items():    
    mean_general_sin_voluntario = None
    num_ficheros = 0
    
    # Calculo de la media general sin el voluntario actual
    for vol, fichs in voluntarios_upv.items():
        if vol!=voluntario:
            for fic in fichs:
                data_file = PATH_UPV+'/'+fic
                result = scipy.io.loadmat(data_file)
                result = np.array(result['Results'])
                if mean_general_sin_voluntario is None:
                    mean_general_sin_voluntario = result
                else:
                    mean_general_sin_voluntario = mean_general_sin_voluntario + result 
                    # Suma de los resultados
                    # Promediar la probabilidad y Promediar el tiempo
                num_ficheros+=len(fichs)
    mean_general_sin_voluntario = mean_general_sin_voluntario/num_ficheros
    
    
    # Calculo de la media del voluntario
    mean_voluntario = None
    for fichero in ficheros:
        data_file = PATH_UPV+'/'+fichero
        result = scipy.io.loadmat(data_file)
        result = np.array(result['Results'])
        if mean_voluntario is None:
            mean_voluntario = result
        else:
            mean_voluntario = mean_voluntario + result 
            # Suma de los resultados
            # Promediar la probabilidad y Promediar el tiempo
    mean_voluntario = mean_voluntario/len(ficheros) # Media de los resultados para un mismo voluntario
    
    # Calculo el rango esperado
    std_general = [np.std(mean_general_sin_voluntario[:,0]), np.std(mean_general_sin_voluntario[:,1])]
    lower_bound = mean_general_sin_voluntario - 2 * np.array(std_general).reshape((1, 2))
    upper_bound = mean_general_sin_voluntario + 2 * np.array(std_general).reshape((1, 2))
    
    # Comparar la media del voluntario actual con la media general y detectar outliers
    # Realiza los gráficos si es outlier
    experimento_outlier = (mean_voluntario < lower_bound) | (mean_voluntario > upper_bound)
    # print("Ambas columnas son outliers\n")
    ambas = np.where(experimento_outlier.all(axis=1))[0]
    # print(ambas) # Ambos son Outlier
    # print("Al menos una son outliers\n")
    al_menos_una = np.where(experimento_outlier)[0]
    # print(al_menos_una) # Al menos un es Outlier

    if (len(ambas)>3):
        print(f"El voluntario {voluntario} es un outlier.")
        print(f"Sus experimentos outliers son: {ambas}.")

El voluntario voluntario_upv_1-_1 es un outlier.
Sus experimentos outliers son: [  1   2   4   5   6   7   8   9  10  12  13  14  15  16  17  18  19  22
  24  25  28  29  30  33  34  35  39  40  44  46  47  49  50  51  54  56
  57  60  64  65  66  67  68  69  72  73  74  75  76  77  79  83  85  86
  87  88  90  91  93  94  95  96  97  98 100 102 103 104 105 106 107 108
 112 116 119 123 124 127 128 132 134 139 141 145 147 148 149 150 151 153
 155 156 157 158 159].
El voluntario voluntario_upv_2-_2 es un outlier.
Sus experimentos outliers son: [  0   1   2   3   4   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  27  28  29  30  31  32  33  34  36  37  38  39
  41  43  44  46  47  49  50  51  52  53  54  55  56  57  58  59  61  62
  63  64  65  66  67  68  69  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  91  92  93  95  96  97  98 100 101 102
 104 105 107 108 109 110 111 112 115 116 117 121 124 125 126 127 128 129
 131 132 133 1

In [5]:
PATH_UGR = "./DatosExperimento1/UGR"

# Leer el archivo Excel en un DataFrame de pandas
df = pd.read_excel(PATH_UGR+'/Exp1_Observadores_UGR.xlsx')
    
if not os.path.exists('Resultados_UGR'):
    os.makedirs('Resultados_UGR')

# Crear un diccionario vacío para almacenar los resultados
voluntarios_ugr = {}

# Iterar sobre cada fila del DataFrame
for idx, row in df.iterrows():
    # Tratamiento de nombre y apellidos para evitar que una misma persona se trate como diferente:
    # Quito espacios, mayusculas, minuscula s, etc
    nombre = unidecode(row['Nombre'].lower())
    apellidos = unidecode(row['Apellidos'].lower())
    fichero = row['Fichero']
    # Crear la clave del diccionario usando el nombre y apellidos
    clave = f"{nombre}-{apellidos}".replace(" ", "")
    
    # Agregar el valor de la columna "Fichero" a la lista correspondiente en el diccionario
    if clave in voluntarios_ugr:
        voluntarios_ugr[clave].append(fichero)
    else:
        voluntarios_ugr[clave] = [fichero]
voluntarios_ugr

{'voluntario_ugr_1-_1': ['ResultsJNDsNoiseFlatBG20220401T131441.mat'],
 'voluntario_ugr_2-_2': ['ResultsJNDsNoiseFlatBG20220317T113653.mat',
  'ResultsJNDsNoiseFlatBG20220407T113228.mat'],
 'voluntario_ugr_3-_3': ['ResultsJNDsNoiseFlatBG20220201T134054.mat',
  'ResultsJNDsNoiseFlatBG20220202T124958.mat'],
 'voluntario_ugr_4-_4': ['ResultsJNDsNoiseFlatBG20220427T114635.mat',
  'ResultsJNDsNoiseFlatBG20220408T103353.mat'],
 'voluntario_ugr_5-_5': ['ResultsJNDsNoiseFlatBG20220401T121203.mat',
  'ResultsJNDsNoiseFlatBG20220323T132705.mat'],
 'voluntario_ugr_6-_6': ['ResultsJNDsNoiseFlatBG20220303T125835.mat',
  'ResultsJNDsNoiseFlatBG20220314T113216.mat'],
 'voluntario_ugr_7-_7': ['ResultsJNDsNoiseFlatBG20220318T133520.mat'],
 'voluntario_ugr_8-_8': ['ResultsJNDsNoiseFlatBG20220411T193944.mat',
  'ResultsJNDsNoiseFlatBG20220420T172846.mat'],
 'voluntario_ugr_9-_9': ['ResultsJNDsNoiseFlatBG20220223T112641.mat',
  'ResultsJNDsNoiseFlatBG20220223T161544.mat'],
 'voluntario_ugr_10-_10': ['Resu

In [6]:
# Variables generales
ruidos = scipy.io.loadmat('ruidosExp1.mat')
ruidos = np.array(ruidos['ruidos'])
minX = np.min(ruidos[:, 2])
maxX = np.max(ruidos[:, 2])
LoopNumber = 16
numberObservations = 10
    
for voluntario, ficheros in voluntarios_ugr.items():    
    mean_general_sin_voluntario = None
    num_ficheros = 0
    
    # Calculo de la media general sin el voluntario actual
    for vol, fichs in voluntarios_ugr.items():
        if vol!=voluntario:
            for fic in fichs:
                data_file = PATH_UGR+'/'+fic
                result = scipy.io.loadmat(data_file)
                result = np.array(result['Results'])
                if mean_general_sin_voluntario is None:
                    mean_general_sin_voluntario = result
                else:
                    mean_general_sin_voluntario = mean_general_sin_voluntario + result 
                    # Suma de los resultados
                    # Promediar la probabilidad y Promediar el tiempo
                num_ficheros+=len(fichs)
    mean_general_sin_voluntario = mean_general_sin_voluntario/num_ficheros
    
    
    # Calculo de la media del voluntario
    mean_voluntario = None
    for fichero in ficheros:
        data_file = PATH_UGR+'/'+fichero
        result = scipy.io.loadmat(data_file)
        result = np.array(result['Results'])
        if mean_voluntario is None:
            mean_voluntario = result
        else:
            mean_voluntario = mean_voluntario + result 
            # Suma de los resultados
            # Promediar la probabilidad y Promediar el tiempo
    mean_voluntario = mean_voluntario/len(ficheros) # Media de los resultados para un mismo voluntario
    
    # Calculo el rango esperado
    std_general = [np.std(mean_general_sin_voluntario[:,0]), np.std(mean_general_sin_voluntario[:,1])]
    lower_bound = mean_general_sin_voluntario - 2 * np.array(std_general).reshape((1, 2))
    upper_bound = mean_general_sin_voluntario + 2 * np.array(std_general).reshape((1, 2))
    
    # Comparar la media del voluntario actual con la media general y detectar outliers
    # Realiza los gráficos si es outlier
    experimento_outlier = (mean_voluntario < lower_bound) | (mean_voluntario > upper_bound)
    # print("Ambas columnas son outliers\n")
    ambas = np.where(experimento_outlier.all(axis=1))[0]
    # print(ambas) # Ambos son Outlier
    # print("Al menos una son outliers\n")
    al_menos_una = np.where(experimento_outlier)[0]
    # print(al_menos_una) # Al menos un es Outlier
    if (len(ambas)>3):
        print(f"El voluntario {voluntario} es un outlier.")
        print(f"Sus experimentos outliers son: {ambas}.")

El voluntario voluntario_ugr_1-_1 es un outlier.
Sus experimentos outliers son: [  1   2   5   8  11  13  15  21  22  23  27  28  30  36  37  40  41  42
  45  46  50  53  54  57  59  63  66  70  71  75  79  81  91  92 100 102
 104 110 111 112 118 120 125 127 130 136 151 153].
El voluntario voluntario_ugr_2-_2 es un outlier.
Sus experimentos outliers son: [  0   1   2   3   4   5   6   8   9  10  11  12  13  14  15  16  17  18
  20  21  22  23  24  25  26  27  28  29  31  33  34  35  36  37  38  39
  40  41  42  43  45  46  47  48  49  50  51  52  53  55  56  58  60  61
  62  64  65  68  71  72  73  74  75  76  78  79  80  81  84  85  86  87
  88  89  90  92  93  94  95  96  98  99 100 101 102 103 106 108 109 110
 112 113 114 117 118 119 121 124 125 126 127 128 129 131 132 133 134 135
 136 137 138 139 141 142 143 144 145 146 147 148 150 151 152 153 154 155
 156 157 158 159].
El voluntario voluntario_ugr_3-_3 es un outlier.
Sus experimentos outliers son: [  0   1   2   4   5   7  12  13 

El voluntario voluntario_ugr_29-_29 es un outlier.
Sus experimentos outliers son: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  48  49  50  51  52  53  54
  55  56  57  58  59  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  97  98  99 100 101 102 103 104 105 106 107 109 110 111 112
 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130
 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 151 152 153 154 155 156 157 158 159].
El voluntario voluntario_ugr_30-_30 es un outlier.
Sus experimentos outliers son: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  21  22  23  24  25  26  27  28  29  30  31  33  34  35  36  37
  38  39  40  41  42  44  45  46  47  48  49  50  51  52  53  5